In [1]:
import pandas as pd
import numpy as np
import pickle
import utils
from collections import Counter

In [2]:
data = pd.read_csv('challenge_dat.csv')

In [3]:
data.head()

,RESTAURANT_NAME,CITY,COUNTRY,R_PRIMARY_LOC_TAG,R_OTHER_LOC_TAG,R_MENU_CATEGORY,R_OTHER_MENU_TAGS,ITEM_NAME,DESCRIPTION,IS_ALCOHOL
0,Hero Sandwich,Hong Kong,Hong Kong,american,western,Sandwiches,"american, western, burgers, Pasta, Smoothies, ...",Corn Beef Baked Potato,鹹牛肉焗薯,False
1,Hamachi-Ya,Sydney,Australia,japanese,NaN,japanese,"sushi, bento, Cream of the crop, Big flavours,...",Spicy Garlic Edamame,Poached green soy beans tossed with salt fresh...,False
2,Hamachi-Ya,Sydney,Australia,japanese,NaN,japanese,"sushi, bento, Cream of the crop, Big flavours,...",Soft Shell Crab (2pcs),Deep-fried soft shell crab with light ponzu sauce,False
3,Hamachi-Ya,Sydney,Australia,japanese,NaN,japanese,"sushi, bento, Cream of the crop, Big flavours,...",Harumaki (4Pcs),Deep-fried Japanese style vegetable springroll,False
4,Hamachi-Ya,Sydney,Australia,japanese,NaN,japanese,"sushi, bento, Cream of the crop, Big flavours,...",Avocado,NaN,False


In [4]:
ascii_data = data.copy()
#remove non ascii from description, and text items
columns_to_ascii = ['R_OTHER_MENU_TAGS','ITEM_NAME','DESCRIPTION']

In [ ]:
ascii_data = ascii_data.dropna(subset=['R_OTHER_MENU_TAGS','DESCRIPTION','ITEM_NAME','R_MENU_CATEGORY','R_PRIMARY_LOC_TAG'])
print(len(ascii_data), ' after na removal')
for col in columns_to_ascii:
    ascii_data[col] = ascii_data[col].astype(str).apply(lambda x: ''.join(['' if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    ascii_data[col].replace('', np.nan, inplace=True)
#remove nan
df = ascii_data.dropna(subset=['R_OTHER_MENU_TAGS','DESCRIPTION','ITEM_NAME','R_MENU_CATEGORY','R_PRIMARY_LOC_TAG'])
print(len(df), ' after ascii')

775109  after na removal
738849  after ascii


In [ ]:
#merging the 3 columns
df['food'] = df['R_OTHER_MENU_TAGS'] + "," + df['ITEM_NAME'] + "," + df['DESCRIPTION']
df = df.drop(columns_to_ascii, axis=1)
df = pd.get_dummies(df, columns=['RESTAURANT_NAME','CITY	COUNTRY	R_PRIMARY_LOC_TAG	R_OTHER_LOC_TAG	R_MENU_CATEGORY	])

/Users/aimihat/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
df = df.loc[:10000]
tok_df = df.copy()
tok_df['food'] = tok_df['food'].apply(utils.tokenize)

In [ ]:
#print(df[df['food'].str.contains('13th')]['food'].tolist())

In [ ]:
all_words = []
for row in tok_df['food']:
    for word in row.split(','):
        all_words.append(word)

In [ ]:
#tok_df['food'].to_csv('10000.csv')

In [ ]:
Counter(all_words).most_common(10)

In [ ]:
#remove unwanted words
generic_words = ['with', 'and', 'the', 'our', 'your']
for word in generic_words:
    if word in all_words:
        all_words.remove(word)
        
word_set = set(all_words)

In [ ]:
encoded_df = tok_df.copy()

In [ ]:
#for everyword
for word in word_set:
    #create column with 1 if appears 0 else
    encoded_df[word] = tok_df['food'].str.contains(word)

In [ ]:
len(encoded_df.columns)